In [ ]:
import multiprocessing
from multiprocessing import Pool

In [ ]:
def custom_pairwise_distances_i_triangle(data, distance_function, i):
    distance_i = [distance_function(data[i], data[j]) for j in range(i)]
    distance_i.append(0)
    return distance_i

def custom_pairwise_distances_triangle_for_multithread(global_data, distance_function, starting_index, length):
    distances = [custom_pairwise_distances_i_triangle(global_data, distance_function, i+starting_index) for i in range(length)]
    return distances

In [ ]:
def get_starting_indices(len_data, divide_into):
    linspace = np.linspace(0, len_data, divide_into)
    indices = [math.floor(x) for x in linspace]
    return indices    

def get_lengths(len_data, indices):
    if len(indices) <= 1:
        return [len_data]
    lengths = [indices[1]]
    for i in range(2, len(indices)):
        lengths.append(indices[i]-indices[i-1])
    lengths.append(len_data-indices[-1])
    return lengths

In [ ]:
def custom_pairwise_distances_triangle_multithread(data, distance_function, divide_into=nb_cpu_cores):
    distances = []    
    starting_indices = get_starting_indices(len(data), divide_into)
    lengths = get_lengths(len(data), starting_indices)
    
    with Pool() as p:
        async_results = [p.apply_async(custom_pairwise_distances_triangle_for_multithread, 
                                       args=(data, distance_function, starting_indices[i], lengths[i])) for i in range(len(lengths))]
        
        for i in range(len(lengths)):
            distances += async_results[i].get()
    return distances
    
def custom_pairwise_distances(data, distance_function, divide_into=nb_cpu_cores):
    distances_triangle = custom_pairwise_distances_triangle_multithread(data, distance_function, divide_into=divide_into)
    for i in range(len(data)):
        for j in range(i+1, len(data)):
            distances_triangle[i].append(distances_triangle[j][i])
    return np.array(distances_triangle)

In [ ]:
def min_distance(distances):
    min_ = float('inf')
    for i in range(len(distances)):
        for j in range(i):
            min_ = min(min_, distances[i][j])
    return min_

def scale_distances(distances):
    min_ = min_distance(distances)
    max_ = np.max(distances)
    for i in range(len(distances)):
        for j in range(len(distances[i])):
            distances[i][j] = (distances[i][j] - min_) / (max_ - min_)
    return